In [ ]:
#vibo: Виталий_Бочаров_stepik_id_56089805, telegram(@VitalyBo)

<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

# Embeddings

Привет! В этом домашнем задании мы с помощью эмбеддингов решим задачу семантической классификации твитов.

Для этого мы воспользуемся предобученными эмбеддингами word2vec.

Для начала скачаем датасет для семантической классификации твитов:

In [ ]:
# #vibo: обновляем gdown (иначе не качал)
# !pip install --upgrade --no-cache-dir gdown

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14774 sha256=1f3b51ce2425de241b5e944a32cf87b972b4016a61126390bb5aed0a667df645
  Stored in directory: /tmp/pip-ephem-wheel-cache-qvo7cm05/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.2.2
    Uninstalling gdown-4.2.2:
      Successfully uninstalled gdown-4.2.2


In [ ]:
# #vibo: скачиваем датасет для семантической классификации твитов
# !gdown https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph&export=download

Downloading...
From: https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph
To: /content/archive.zip
100% 84.9M/84.9M [00:00<00:00, 131MB/s]


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#vibo: разархивируем
!unzip '/content/drive/MyDrive/Colab Notebooks/archive.zip'

Archive:  /content/drive/MyDrive/Colab Notebooks/archive.zip
  inflating: training.1600000.processed.noemoticon.csv  


Импортируем нужные библиотеки:

In [3]:
#vibo: импортируемые библиотеки
import math
import random
import string

import numpy as np
import pandas as pd
import seaborn as sns

import torch
import nltk
import gensim
import gensim.downloader as api

In [4]:
#vibo: генератор случайных чисел
random.seed(42)
#vibo: генератор случайных чисел
np.random.seed(42)
#vibo: начальное число для генерации случайных чисел.
torch.random.manual_seed(42)
#vibo: устанавливает начальное число для генерации случайных чисел для текущего графического процессора.
#vibo: эту функцию безопасно вызывать, если CUDA недоступен; в этом случае он молча игнорируется.
torch.cuda.random.manual_seed(42)
#vibo: задает начальное значение для генерации случайных чисел на всех графических процессорах.
#vibo: эту функцию безопасно вызывать, если CUDA недоступен; в этом случае он молча игнорируется.
torch.cuda.random.manual_seed_all(42)

#vibo: выбор доступного девайса
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
#vibo: загружаем данные
data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin", header=None, names=["emotion", "id", "date", "flag", "user", "text"])

Посмотрим на данные:

In [6]:
data.shape

(1600000, 6)

In [7]:
data.head()

,emotion,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


Выведем несколько примеров твитов, чтобы понимать, с чем мы имеем дело:

In [8]:
examples = data["text"].sample(10)
print("\n".join(examples))

@chrishasboobs AHHH I HOPE YOUR OK!!! 
@misstoriblack cool , i have no tweet apps  for my razr 2
@TiannaChaos i know  just family drama. its lame.hey next time u hang out with kim n u guys like have a sleepover or whatever, ill call u
School email won't open  and I have geography stuff on there to revise! *Stupid School* :'(
upper airways problem 
Going to miss Pastor's sermon on Faith... 
on lunch....dj should come eat with me 
@piginthepoke oh why are you feeling like that? 
gahh noo!peyton needs to live!this is horrible 
@mrstessyman thank you glad you like it! There is a product review bit on the site  Enjoy knitting it!


Как видим, тексты твитов очень "грязные". Нужно предобработать датасет, прежде чем строить для него модель классификации.

Чтобы сравнивать различные методы обработки текста/модели/прочее, разделим датасет на dev(для обучения модели) и test(для получения качества модели).

In [9]:
import numpy as np

#vibo: делаем нумерацию всех твитов
indexes = np.arange(data.shape[0])
#vibo: перемешиваем индексы
np.random.shuffle(indexes)

In [10]:
import math
#vibo: берем 80% на обучение
#vibo: math.ceil - округление вверх до ближайшего большего целого
dev_size = math.ceil(data.shape[0] * 0.8)

In [11]:
#vibo: размер обучающего датасета
dev_size

1280000

In [12]:
#vibo: задаем индексацию объектов для обучения и тестирования
dev_indexes = indexes[:dev_size]
test_indexes = indexes[dev_size:]

# !!! dev_data (0.8data), test_data(0.2data) 

In [13]:
#vibo: задаем обучающий и тестовый датасет
#vibo: iloc - индексация по индексам
dev_data = data.iloc[dev_indexes]
test_data = data.iloc[test_indexes]

In [14]:
#vibo: датасет для обучения
dev_data

,emotion,id,date,flag,user,text
33408,0,1564500154,Mon Apr 20 03:47:08 PDT 2009,NO_QUERY,zourzouvillys,@Claire_Nelson i'm on the north devon coast th...
1041124,4,1957039896,Thu May 28 23:21:01 PDT 2009,NO_QUERY,twinkleval,@jhicks i will think of you on Sunday! Who ...
832491,4,1557601862,Sun Apr 19 05:03:53 PDT 2009,NO_QUERY,andrewbulloch,Out in the garden with the kids debating wheth...
948959,4,1823599026,Sat May 16 22:25:01 PDT 2009,NO_QUERY,vikchopra,@FrVerona thank u my love...u've shown me the ...
1562417,4,2186814798,Mon Jun 15 19:22:35 PDT 2009,NO_QUERY,jobrofan16,is with @jonasbrosfan1 going to buy LVATT tog...
...,...,...,...,...,...,...
1141780,4,1977221349,Sat May 30 20:25:09 PDT 2009,NO_QUERY,omgxregina,@yajtyler thats so sweet =] they like you and...
478676,0,2178617823,Mon Jun 15 07:57:56 PDT 2009,NO_QUERY,ilykthx,Really sad the NBA playoffs are over for good
1424044,4,2058796673,Sat Jun 06 15:38:43 PDT 2009,NO_QUERY,Socallmelovely,@JuJuBeanz15 LOL Im glad that u have that pers...
131963,0,1835648576,Mon May 18 06:27:33 PDT 2009,NO_QUERY,tercowas,I have to go to the GYM it's toooooo hot.


In [15]:
#vibo: сбрасываем индексацию
dev_data.reset_index(drop=True, inplace=True)
dev_data

,emotion,id,date,flag,user,text
0,0,1564500154,Mon Apr 20 03:47:08 PDT 2009,NO_QUERY,zourzouvillys,@Claire_Nelson i'm on the north devon coast th...
1,4,1957039896,Thu May 28 23:21:01 PDT 2009,NO_QUERY,twinkleval,@jhicks i will think of you on Sunday! Who ...
2,4,1557601862,Sun Apr 19 05:03:53 PDT 2009,NO_QUERY,andrewbulloch,Out in the garden with the kids debating wheth...
3,4,1823599026,Sat May 16 22:25:01 PDT 2009,NO_QUERY,vikchopra,@FrVerona thank u my love...u've shown me the ...
4,4,2186814798,Mon Jun 15 19:22:35 PDT 2009,NO_QUERY,jobrofan16,is with @jonasbrosfan1 going to buy LVATT tog...
...,...,...,...,...,...,...
1279995,4,1977221349,Sat May 30 20:25:09 PDT 2009,NO_QUERY,omgxregina,@yajtyler thats so sweet =] they like you and...
1279996,0,2178617823,Mon Jun 15 07:57:56 PDT 2009,NO_QUERY,ilykthx,Really sad the NBA playoffs are over for good
1279997,4,2058796673,Sat Jun 06 15:38:43 PDT 2009,NO_QUERY,Socallmelovely,@JuJuBeanz15 LOL Im glad that u have that pers...
1279998,0,1835648576,Mon May 18 06:27:33 PDT 2009,NO_QUERY,tercowas,I have to go to the GYM it's toooooo hot.


In [16]:
#vibo: выборка сбалансирована
from collections import Counter
Counter(list(dev_data['emotion']))

Counter({0: 640028, 4: 639972})

In [17]:
test_data.reset_index(drop=True, inplace=True)
test_data

,emotion,id,date,flag,user,text
0,0,1956414820,Thu May 28 21:49:47 PDT 2009,NO_QUERY,Miss_LeAnn,eyes getting heavy... dont think im going to m...
1,4,1556092230,Sat Apr 18 21:37:13 PDT 2009,NO_QUERY,theIntriguing1,@cathy_nutrition Ok..that's fine.
2,4,1882430417,Fri May 22 06:51:32 PDT 2009,NO_QUERY,AJgeorge,@lucysenior oh and mechanics wasn't that great...
3,4,2065266715,Sun Jun 07 08:14:28 PDT 2009,NO_QUERY,franci_92,LALALA....... Im going to New York visiting my...
4,4,1990167622,Mon Jun 01 04:05:16 PDT 2009,NO_QUERY,siumuimui,had a nice coffee break and chat with @alexcg!...
...,...,...,...,...,...,...
319995,0,2324335179,Thu Jun 25 02:54:12 PDT 2009,NO_QUERY,twittlight,@HusseinNasser pls pls pls help this VB idiot ...
319996,4,1971122047,Sat May 30 06:16:33 PDT 2009,NO_QUERY,NurseForHealth,@curescancer Lol! You are right. Thank you
319997,0,2203265388,Tue Jun 16 23:21:53 PDT 2009,NO_QUERY,kato0512,Ugh feel so sick I can't move don't think I c...
319998,0,1883462165,Fri May 22 08:35:22 PDT 2009,NO_QUERY,xCazziex,"Like my new form, but not sure i want to split..."


In [18]:
#vibo: т.е. меток у нас всего две (думал, что больше)
set(dev_data['emotion'])

{0, 4}

## Обработка текста

Токенизируем текст, избавимся от знаков пунктуации и выкинем все слова, состоящие менее чем из 4 букв:

In [ ]:
#vibo: индексация в pandas объект['имя_столбца'][индекс_строки]
dev_data["text"][1]

'@jhicks  i will think of you on Sunday!   Who are you by the way?  i am Val.'

In [ ]:
#vibo: пробую удалить имена пользователей и адреса сайтов
simple_str = ("@actheboss your http://...eaerng;iea check can never be fatter than mine.... LOL you'd have to stay http://........ @ the job overnight until the end of summer... ")
simple_str = simple_str.split(' ')
simple_str = (' '.join(filter(lambda word: not (word.lower().startswith('@') or word.lower().startswith('http')), simple_str)))
simple_str

"your check can never be fatter than mine.... LOL you'd have to stay the job overnight until the end of summer... "

In [ ]:
#vibo: разбиваем тестовую строку на токены, предвариетльно приведя ее к нижнему регистру
tokenizer = nltk.WordPunctTokenizer()
line = tokenizer.tokenize(dev_data["text"][0].lower())
print(" ".join(line))

@ claire_nelson i ' m on the north devon coast the next few weeks will be down in devon again in may sometime i hope though !


In [ ]:
print(line)

['@', 'claire_nelson', 'i', "'", 'm', 'on', 'the', 'north', 'devon', 'coast', 'the', 'next', 'few', 'weeks', 'will', 'be', 'down', 'in', 'devon', 'again', 'in', 'may', 'sometime', 'i', 'hope', 'though', '!']


In [ ]:
#vibo: удаляем знаки пунктуации и оставляем слова 4 буквы и больше
filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
print(" ".join(filtered_line))

north devon coast next weeks will down devon again sometime hope though


In [ ]:
print(filtered_line)

['north', 'devon', 'coast', 'next', 'weeks', 'will', 'down', 'devon', 'again', 'sometime', 'hope', 'though']


In [ ]:
#vibo: имена я бы тоже убрал... и ссылки

Загрузим предобученную модель эмбеддингов. 

Если хотите, можно попробовать другую. Полный список можно найти здесь: https://github.com/RaRe-Technologies/gensim-data.

Данная модель выдает эмбеддинги для **слов**. Строить по эмбеддингам слов эмбеддинги предложений мы будем ниже.

In [ ]:
import gensim.downloader as api

#vibo: загружаем предобученные эмбединги слов
word2vec = api.load("word2vec-google-news-300")
#vibo: спасибо чату https://t.me/dls_spring2022_part2
#word2vec = api.load('glove-twitter-50')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
#vibo: получим векторные представления для каждого слова из тестовой строки
emb_line = [word2vec.get_vector(w) for w in filtered_line if w in word2vec]
print(sum(emb_line).shape)

(300,)


Нормализуем эмбеддинги, прежде чем обучать на них сеть. 
(наверное, вы помните, что нейронные сети гораздо лучше обучаются на нормализованных данных)

In [ ]:
#vibo: в словаре, который мы скачали 3_000_000 слов
(word2vec.vectors).shape

(3000000, 300)

In [ ]:
#vibo: так можно посмотреть слово
(word2vec['dog']).shape

(300,)

In [ ]:
mean = np.mean(word2vec.vectors, 0)
std = np.std(word2vec.vectors, 0)

In [ ]:
norm_emb_line = [(word2vec.get_vector(w) - mean) / std for w in filtered_line if w in word2vec and len(w) > 3]
print(sum(norm_emb_line).shape)
print([all(norm_emb_line[i] == emb_line[i]) for i in range(len(emb_line))])

(300,)
[False, False, False, False, False, False, False, False, False, False, False, False]


Сделаем датасет, который будет по запросу возвращать подготовленные данные.

# !!! TwitterDataset

In [ ]:
import pandas as pd
from torch.utils.data import Dataset, random_split
import gensim

#vibo: делаем класс, который будет возвращать эмбединги для каждого твита датасета
class TwitterDataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        #vibo: определяем токенайзер
        self.tokenizer = nltk.WordPunctTokenizer()
        
        #vibo: на вход подается датафрейм
        self.data = data

        #vibo: на вход подается колонка c текстом твива 'text'
        self.feature_column = feature_column
        #vibo: на вход подается колонка с метокой эмоции 'emotion'
        self.target_column = target_column

        #vibo: для векторизации будем использовать предобученные эмбединги загружали их выше
        self.word2vec = word2vec

        #vibo: исходные метки твитов 0 и 4, здесь их меняем на 0 и 1
        self.label2num = lambda label: 0 if label == 0 else 1
        #vibo: для нормализации ниже 
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        #vibo: получаем текст твита по заданному item (номер строки)
        text = self.data[self.feature_column][item]
        #vibo: получаем метку эмоции твита по заданному item (номер строки)
        label = self.label2num(self.data[self.target_column][item])

        #vibo: здесь будут tokens после реалиации функции get_tokens_
        tokens = self.get_tokens_(text)
        #vibo: здесь будут embeddings реалиации функции get_embeddings_
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
       
        #vibo: не влияет на результат
        # #vibo: убираем имена пользователей и ссылки на сайты
        # text = text.split(' ')
        # text = (' '.join(filter(lambda word: not (word.lower().startswith('@') or word.lower().startswith('http')), text)))

        #vibo: полученную строку пропускаем через токенайзер, предварительно переведя в нижний регистр
        line = self.tokenizer.tokenize(text.lower())
        #print('line:', line)
        #vibo: удаляем знаки пунктуации и слова короче 4 букв
        tokens = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
        #vibo: возвращаем токены
        return  tokens

    def get_embeddings_(self, tokens):
       
        #print('tokens:', tokens)
        #vibo: получаем эмбединги слов из предобученной библиотеки и нормируем их
        embeddings = [(self.word2vec.get_vector(w) - self.mean) / self.std for w in tokens if w in self.word2vec] # Получи эмбеддинги слов и усредни их      
        #print('embeddings:', len(embeddings))

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        # #vibo: попринтим
        # for i in embeddings:
        #   print(i)
        #   print(type(i))
        
        # print('embeddings:', embeddings)
        # print('type embeddings:', type(embeddings))
        # print('shape embeddings:', embeddings.shape)
        return embeddings

    def __len__(self):
        return self.data.shape[0]

# !!! Обучающий датасет dev (dev_data)

In [ ]:
#vibo: получаем обработанный датасет
'''
dev_data - дафрейм для обучения dev_data
"text" - наименование столбца с твитами
"emotion" - наименование столбца с меткой класса
word2vec - предобученные эмбединги слов
'''

dev = TwitterDataset(dev_data, "text", "emotion", word2vec)

In [ ]:
for i in range(0, 27):
  print(i)
  print((dev[i]["feature"]).shape)
  #print((dev[i]["feature"]))

In [ ]:
dev_data["text"][333]

"@newcastleblog this is a brilliant idea, can't wait til its all up and running "

In [ ]:
#vibo: проверка твитов (матрица с эмбедингами слов, входящих в твит)
(dev[33000]["feature"]).shape

(10, 300)

In [ ]:
#vibo: целевая метка для соответствующего твита
dev[333]["target"]

1

Отлично, мы готовы с помощью эмбеддингов слов превращать твиты в векторы и обучать нейронную сеть.

Превращать твиты в векторы, используя эмбеддинги слов, можно несколькими способами. А именно такими:

## Average embedding (2 балла)
---
Это самый простой вариант, как получить вектор предложения, используя векторные представления слов в предложении. А именно: вектор предложения есть средний вектор всех слов в предложнии (которые остались после токенизации и удаления коротких слов, конечно). 

In [ ]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

#vibo: исправил sum на mean, спасибо чату https://t.me/dls_spring2022_part2
#vibo: здесь для выборочного примера усредняем эмбединги слов по строке твита
examples = {"features": [np.mean(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1280


In [ ]:
(examples["features"][0]).shape

(300,)

In [ ]:
set(examples["targets"])

{0, 1}

Давайте сделаем визуализацию полученных векторов твитов тренировочного (dev) датасета. Так мы увидим, насколько хорошо твиты с разными target значениями отделяются друг от друга, т.е. насколько хорошо усреднение эмбеддингов слов предложения передает информацию о предложении.

Для визуализации векторов надо получить их проекцию на плоскость. Сделаем это с помощью `PCA`. Если хотите, можете вместо PCA использовать TSNE: так у вас получится более точная проекция на плоскость (а значит, более информативная, т.е. отражающая реальное положение векторов твитов в пространстве). Но TSNE будет работать намного дольше.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)

#vibo: обучаем PCA на полученных примерах
examples["transformed_features"] = pca.fit_transform(examples["features"]) # Обучи PCA на эмбеддингах слов
#examples["transformed_features"] = pca.fit_transform(np.transpose(examples["features"]))

In [ ]:
examples.keys()

dict_keys(['features', 'targets', 'transformed_features'])

In [ ]:
len(examples['features'][0])

300

In [ ]:
examples['targets'][0]

0

In [ ]:
examples['transformed_features'][0]

array([-1.35587883, -0.28357077])

In [ ]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [ ]:
%matplotlib inline
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Figure(id='1003', ...)

Скорее всего, на визуализации нет четкого разделения твитов между классами. Это значит, что по полученным нами векторам твитов не так-то просто определить, к какому классу твит пренадлежит. Значит, обычный линейный классификатор не очень хорошо справится с задачей. Надо будет делать глубокую (хотя бы два слоя) нейронную сеть.

Подготовим загрузчики данных.
Усреднее векторов будем делать в "батчевалке"(`collate_fn`). Она используется для того, чтобы собирать из данных `torch.Tensor` батчи, которые можно отправлять в модель.


# !!! train_loader (0.8dev), valid_loader (0.2dev)

In [ ]:
from torch.utils.data import DataLoader

batch_size = 1024
#vibo: спасибо чату https://t.me/dls_spring2022_part2
#num_workers = 4

#vibo: усредняем эмбединг по строке
def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}

train_size = math.ceil(len(dev) * 0.8)
#vibo: делим обучающий датасет на train и valid 
train, valid = random_split(dev, [train_size, len(dev) - train_size])

# train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
# valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

#vibo: внимательно для train_loader shuffle=True
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, drop_last=True, collate_fn=average_emb)
#vibo: внимательно для valid_loader shuffle=False
valid_loader = DataLoader(valid, batch_size=batch_size, shuffle=False, drop_last=False, collate_fn=average_emb)

In [ ]:
for a, b in train_loader:
  print(a, b)
  break

features targets


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  if sys.path[0] == '':


Определим функции для тренировки и теста модели:

# !!! training & testing

In [ ]:
from tqdm.notebook import tqdm
#vibo: импортирую accuracy
from sklearn.metrics import accuracy_score

def training(model, optimizer, criterion, train_loader, epoch, device="cpu"):

    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}")
    model.train()
    for batch in pbar:
        features = batch["features"].to(device)
        targets = batch["targets"].to(device)

        # Получи предсказания модели     
        #vibo: обнуляем градиенты модели
        optimizer.zero_grad()
        #vibo: получаем предсказания модели, убираем лишнюю размерность батча
        outp = model(features).view(-1)
        #vibo: считаем loss, приводим метки класса к float
        loss = criterion(outp, targets.float()) # Посчитай лосс

        # Обнови параметры модели
        #vibo: считаем градиенты
        loss.backward()
        #vibo: делаем шаг градиентного спуска
        optimizer.step()

        pbar.set_description(f"Epoch {e + 1}. Train Loss: {loss:.4}")
    

def testing(model, criterion, test_loader, device="cpu"):

    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].to(device)

            # Получи предсказания модели           
            #vibo: получаем предсказания модели, убираем лишнюю размерность батча
            outp = model(features).view(-1)
            #vibo: считаем loss, приводим метки класса к float
            loss = criterion(outp, targets.float()) # Посчитай лосс               
            #vibo: от полученных на выходе модели логитов переходим к вероятности
            outp_prob = torch.sigmoid(outp)  
            #vibo: переводим полученные вероятности в бинарные метки
            outp_pred = (outp_prob > 0.5).type(torch.long)
            #vibo: считаем accuracy
            acc = accuracy_score(targets, outp_pred) # Посчитай точность модели

            mean_loss += loss.item()
            mean_acc += acc.item()

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, Test Acc: {mean_acc / len(test_loader):.4}")

    #vibo: делаем словарь с двумя ключами "Test Loss" и "Test Acc"
    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}

Создадим модель, оптимизатор и целевую функцию. Вы можете сами выбрать количество слоев в нейронной сети, ваш любимый оптимизатор и целевую функцию.


# !!! М О Д Е Л Ь + результаты (изменяемые_параметры), criterion, optimizer

In [ ]:
import torch.nn as nn
from torch.optim import Adam


# Не забудь поиграться с параметрами ;)
vector_size = dev.word2vec.vector_size

#vibo: количество нейронов выходного слоя
#num_classes = 2
num_classes = 1

#vibo: learning rate для Adam/AdamW
#lr = 1e-2
lr = 3e-4

#vibo: поменял количество эпох обучения
#num_epochs = 1
num_epochs = 5

#vibo: простая двухслойная медель
model = nn.Sequential( # Твоя модель
        #vibo: первый линейный слой - на вход vector_size
        nn.Linear(vector_size, 128),
        #vibo: функция активации, релу
        nn.ReLU(),
        nn.Linear(128, 128),
        #vibo: функция активации, релу
        nn.ReLU(),
        #vibo: последний линейный слой
        nn.Linear(128, num_classes)
)

'''
==ДЛЯ ПРЕДОБУЧЕННЫХ ЭМБЕДДИНГОВ==

#vibo: если УДАЛИТЬ имена пользователей и ссылки http

      Adam, lr = 3e-4, один промежуточный слой с 8 нейронами
      Полный log (на валидации): {'Test Loss': 0.5333307747840881, 'Test Acc': 0.73254296875}

      Adam, lr = 3e-4, один промежуточный слой с 16 нейронами
      Полный log (на валидации): {'Test Loss': 0.5290642886161804, 'Test Acc': 0.7353515625}

      Adam, lr = 3e-4, один промежуточный слой с 32 нейронами
      Полный log (на валидации): {'Test Loss': 0.520956760764122, 'Test Acc': 0.74246875}

      Adam, lr = 3e-4, один промежуточный слой с 64 нейронами
      Полный log (на валидации): {'Test Loss': 0.5133036247491837, 'Test Acc': 0.746484375}

      Adam, lr = 3e-4, один промежуточный слой с 128 нейронами
      Полный log (на валидации): {'Test Loss': 0.5074361897706986, 'Test Acc': 0.74965625}

                    Adam, lr = 3e-4, ДВА промежуточных слоя с 128 нейронами
                    Полный log (на валидации): {'Test Loss': 0.4973389925956726, 'Test Acc': 0.7555_234375}

      Adam, lr = 3e-4, ДВА промежуточных слоя с 64 нейронами
      Полный log (на валидации): {'Test Loss': 0.5043212249279022, 'Test Acc': 0.751}
'''

'''
#vibo: если НЕ УДАЛЯТЬ имена пользователей и ссылки http

      Adam, lr = 3e-4, ДВА промежуточных слоя с 64 нейронами
      Полный log (на валидации): {'Test Loss': 0.5043130439519882, 'Test Acc': 0.7504453125}

      AdamW, lr = 3e-4, ДВА промежуточных слоя с 64 нейронами
      Полный log (на валидации): {'Test Loss': 0.5040333566665649, 'Test Acc': 0.751125}

      AdamW, lr = 1e-2,  ДВА промежуточных слоя с 64 нейронами
      Полный log (на валидации): {'Test Loss': 0.5008773572444916, 'Test Acc': 0.75294140625}

                    AdamW, lr = 1e-2,  ДВА промежуточных слоя с 128 нейронами
                    Полный log (на валидации): {'Test Loss': 0.5016085507869721, 'Test Acc': 0.75265234375}

                    AdamW, lr = 3e-4,  ДВА промежуточных слоя с 128 нейронами
                    Полный log (на валидации): {'Test Loss': 0.49861656868457793, 'Test Acc': 0.7552_1484375}

                                  Adam, lr = 3e-4,  ДВА промежуточных слоя с 128 нейронами (10 эпох)
                                  Полный log (на валидации): {'Test Loss': 0.49392504930496217, 'Test Acc': 0.7575_8984375}
                                  Полный log (на тесте):     {'Test Loss': 0.4940757559130367,  'Test Acc': 0.7581_52580870607}
'''
'''
==ДЛЯ ПРЕДОБУЧЕННЫХ ЭМБЕДДИНГОВ и заменой слов не из словаря суммой их контекста (+-2 слова)==

                                  Adam, lr = 3e-4,  ДВА промежуточных слоя с 128 нейронами (5 эпох)
                                  Полный log (на валидации): {'Test Loss': 0.5019097927808761, 'Test Acc': 0.7466_5234375}
                                  Полный log (на тесте): {'Test Loss': 0.5076717542002376, 'Test Acc': 0.7431_047823482428}

'''


#vibo: модель тоже нужно отправить на device
#model = model.cuda()
model = model.to(device)

#vibo: задаем функцию потерь
criterion = nn.BCEWithLogitsLoss() # Твой лосс
#vibo: задаем оптимайзер
optimizer = Adam(model.parameters(), lr=lr) # Твой оптимайзер

In [ ]:
#vibo: обзор слоев модели
model

Sequential(
  (0): Linear(in_features=300, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=1, bias=True)
)

In [ ]:
#vibo: использование torchsummary
from torchsummary import summary

#vibo: обзор слове модели с параметрами
summary(model, (vector_size,), batch_size)

#vibo: или так 
#summary(model, (batch_size,vector_size))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [1024, 128]          38,528
              ReLU-2                [1024, 128]               0
            Linear-3                [1024, 128]          16,512
              ReLU-4                [1024, 128]               0
            Linear-5                  [1024, 1]             129
Total params: 55,169
Trainable params: 55,169
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 1.17
Forward/backward pass size (MB): 4.01
Params size (MB): 0.21
Estimated Total Size (MB): 5.39
----------------------------------------------------------------


Наконец, обучим модель и протестируем её.

После каждой эпохи будем проверять качество модели на валидационной части датасета. Если метрика стала лучше, будем сохранять модель. **Подумайте, какая метрика (точность или лосс) будет лучше работать в этой задаче?** 

#vibo: правильнее использовать loss

# !!! ОБУЧЕНИЕ МОДЕЛИ

In [ ]:
#vibo: здесь обучаем и валидируем модель на dev(dev_data): train_loader (0.8dev) и valid_loader (0.2dev)
#vibo: ПОДБИРАЕМ ПАРАМЕТРЫ МОДЕЛИ

#vibo: бесконечность...
best_metric = np.inf

#vibo: проходим по эпохам
for e in range(num_epochs):
    #vibo: обучение, функция training на train_loader -> 0.8dev(dev_data) 
    training(model, optimizer, criterion, train_loader, e, device)
    #vibo: оцениваем модель, функция testing на valid_loader -> 0.2dev(dev_data)
    log = testing(model, criterion, valid_loader, device)
    print('Полный log (на валидации):', log)
    #vibo: сравниваем результат по loss, сохраняем лучшую модель
    if log["Test Loss"] < best_metric:
        
        
        print('Текущий Test Loss:', log["Test Loss"])
        print('Предыдущий Test Loss:', best_metric)
        
        #vibo: сохраняем модель
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]
        print('Новый Test Loss:', best_metric)
        print('')

Epoch 1. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

Полный log (на валидации): {'Test Loss': 0.518032349228859, 'Test Acc': 0.74144921875}
Текущий Test Loss: 0.518032349228859
Предыдущий Test Loss: inf
Новый Test Loss: 0.518032349228859



Epoch 2. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

...

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

Полный log (на валидации): {'Test Loss': 0.49333326268196104, 'Test Acc': 0.75878515625}
Текущий Test Loss: 0.49333326268196104
Предыдущий Test Loss: 0.49529907655715943
Новый Test Loss: 0.49333326268196104



Epoch 10. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

Полный log (на валидации): {'Test Loss': 0.49392504930496217, 'Test Acc': 0.75758984375}


# !!! Тестовый датасет, test_loader (test_data)

In [ ]:
#vibo: здесь оцениваем результат лучшей модели  на отложенных данных test_data(0.2data)
#vibo: ОЦЕНИВАЕМ ИТОГОВЫЙ РЕЗУЛЬТАТ

#vibo: получаем тестовый датасет (test_loader)
#vibo: внимательно для test_loader shuffle=False
#vibo: collate_fn=average_emb, average_emb - функция усреднения эмбедингов, base-line выше
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    #num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

#vibo: загружаем лучшую сохраненную модель
model.load_state_dict(torch.load("model.pt", map_location=device))

#vibo: оцениваем модель, функция testing на test_loader -> test_data
print('Полный log (на тесте):', testing(model, criterion, test_loader, device=device))

Test Loss: 0, Test Acc: 0:   0%|          | 0/313 [00:00<?, ?it/s]

Полный log (на тесте): {'Test Loss': 0.4940757559130367, 'Test Acc': 0.758152580870607}


# !!! ПРОВЕРКА НА ЗАГРУЖЕННОЙ С ДИСКА МОДЕЛИ

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
#vibo: здесь оцениваем результат лучшей модели  на отложенных данных test_data(0.2data)
#vibo: ОЦЕНИВАЕМ ИТОГОВЫЙ РЕЗУЛЬТАТ

#vibo: получаем тестовый датасет (test_loader)
#vibo: внимательно для test_loader shuffle=False
#vibo: collate_fn=average_emb, average_emb - функция усреднения эмбедингов, base-line выше
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    #num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

#vibo: загружаем лучшую сохраненную модель
model.load_state_dict(torch.load("model.pt", map_location=device))

#vibo: оцениваем модель, функция testing на test_loader -> test_data
print('Полный log (на тесте):', testing(model, criterion, test_loader, device=device))

Test Loss: 0, Test Acc: 0:   0%|          | 0/313 [00:00<?, ?it/s]

Полный log (на тесте): {'Test Loss': 0.4940757559130367, 'Test Acc': 0.758152580870607}


## Embeddings for unknown words (8 баллов)

Пока что использовалась не вся информация из текста. Часть информации фильтровалось – если слова не было в словаре эмбеддингов, то мы просто превращали слово в нулевой вектор. Хочется использовать информацию по-максимуму. Поэтому рассмотрим другие способы обработки слов, которых нет в словаре. А именно:

- Для каждого незнакомого слова будем запоминать его контекст(слова слева и справа от этого слова). Эмбеддингом нашего незнакомого слова будет сумма эмбеддингов всех слов из его контекста. (4 балла)
- Для каждого слова текста получим его эмбеддинг из Tfidf с помощью ```TfidfVectorizer``` из [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer). Итоговым эмбеддингом для каждого слова будет сумма двух эмбеддингов: предобученного и Tfidf-ного. Для слов, которых нет в словаре предобученных эмбеддингов, результирующий эмбеддинг будет просто полученный из Tfidf. (4 балла)

Реализуйте оба варианта **ниже**. Напишите, какой способ сработал лучше и ваши мысли, почему так получилось.

# !!! ЗАДАНИЕ-2_NewTwitterDataset

In [ ]:
# #vibo: делаем новый класс от TwitterDataset, меняю get_embeddings_
# class NewTwitterDataset(TwitterDataset):
#         #vibo: контсруктор класса как у родителя
#         def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
#           #vibo: унаследуем все параметры от родителя
#           super().__init__(data, feature_column, target_column, word2vec)
      
#         #vibo: уточним get_tokens_
#         def get_tokens_(self, text):
#           # Получи все токены из текста и профильтруй их       

#           #vibo: убираем имена пользователей и ссылки на сайты
#           text = text.split(' ')
#           text = (' '.join(filter(lambda word: not (word.lower().startswith('@') or word.lower().startswith('http')), text)))

#           #vibo: полученную строку пропускаем через токенайзер, предварительно переведя в нижний регистр
#           line = self.tokenizer.tokenize(text.lower())
#           #vibo: удаляем знаки пунктуации и слова короче 4 букв
#           tokens = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
#           #vibo: возвращаем токены
#           return tokens

#         #vibo: уточним get_embeddings_
#         def get_embeddings_(self, tokens):
        
#           embeddings = []
#           for w in tokens:
#             #if w in self.word2vec:
#             try:
#               word_embeddings = (self.word2vec.get_vector(w) - self.mean) / self.std
#               embeddings.append(word_embeddings) 
#               #embeddings.append(np.array(word_embeddings))
#             except:
#               embeddings.append('no_embeddings')
#           #print('embeddings:', embeddings)

#           if len(embeddings) == 0:
#               embeddings = np.zeros((1, self.word2vec.vector_size))
#           else:
#               embeddings = np.array(embeddings)
#               if len(embeddings.shape) == 1:
#                   embeddings = embeddings.reshape(-1, 1)

#           for n in range(len(embeddings)):
#             context = []
#             #print('n:', n)
#             #vibo: итерируемся по предложению
#             #vibo: находим слово, которого нет в словаре (ранее его менял на 'no_embeddings')
#             if embeddings[n] == 'no_embeddings':
              
#               #vibo: проверяем есть ли второе слева контекствное слово
#               try:
#                 if (n-2) >= 0:
#                   context_word_n2_left =  embeddings[n-2]
#                   if context_word_n2_left != 'no_embeddings':
#                     context.append(context_word_n2_left)                  
#                 else:
#                   pass
#               except:
#                 pass
            
#               #vibo: проверяем есть ли первое слева контекствное слово
#               try:
#                 if (n-1) >=0:
#                   context_word_n1_left = embeddings[n-1]
#                   if context_word_n1_left != 'no_embeddings':
#                     context.append(context_word_n1_left)
#                 else:
#                   pass
#               except:
#                 pass
              
#               #vibo: проверяем есть ли первое справа контекствное слово
#               try:
#                 context_word_n1_right = embeddings[n+1]
#                 if context_word_n1_right != 'no_embeddings':
#                   context.append(context_word_n1_right)
#               except:
#                 pass
              
#               #vibo: проверяем есть ли второе справа контекствное слово
#               try:
#                 context_word_n2_right = embeddings[n+2]
#                 if context_word_n2_right != 'no_embeddings':
#                   context.append(context_word_n2_right)
#               except:
#                 pass
              
#               #print('context:', context)
#               embeddings[n] = sum(context)
#               #print('sum(context) или embeddings[n]:', embeddings[n])

#           # #vibo: НЕ ПРАВИЛЬНЫЙ РАЗМЕР ЭМБЕДИНГОВ без добавления
#           # #print('NEW embeddings:', embeddings)
#           # embeddings = [np.squeeze(embeddings[i][0]) for i in range(len(embeddings))]
#           # embeddings = np.array(embeddings, dtype=np.float32)
#           # #print('NEW NEW embeddings:', embeddings)

#               #vibo: неправильный размер у предложения с двумя исклчениями
#               #print('NEW embeddings:', embeddings)
#               embeddings = [np.squeeze(embeddings[i][0]) for i in range(len(embeddings))]
#               embeddings = np.array(embeddings, dtype=np.float32)
#               #print('NEW NEW embeddings:', embeddings)



# #vibo: УДАЛИТЬ

#               # #vibo: очередной костыль
#               # #vibo: сумма эмбеддингов соседних слов
#               # #print('total_context:', context)
#               # try:
#               #   embeddings[n] = sum(context)
#               # except:
#               #   pass

#               # #vibo: убираю лишнюю ось
#               # embeddings = np.squeeze(embeddings)
#               # for i in embeddings:
#               #   print(i)
#               #   print(type(i))
              
#               #vibo: может все же среднее???
#               #vibo: при суммировании эмбедингов это слово сильно выбивается из строки
#               #embeddings[n] = mean(context)               
                   
#           # print('embeddings:', embeddings)
#           # print('type embeddings:', type(embeddings))
#           # print('shape embeddings:', embeddings.shape)
          
#           # #vibo: навставлял костыли
#           # embeddings = [np.squeeze(embeddings[i][0]) for i in range(len(embeddings))]
#           # #print('embeddings', embeddings)
#           # embeddings = np.array(embeddings, dtype=np.float32)
#           # # print('type embeddings:', type(embeddings))
#           # # print('shape embeddings:', embeddings.shape)


# #vibo: тоже удалить

          
#           # #print('NEW embeddings:', embeddings)
#           # embeddings = [np.squeeze(embeddings[i][0]) for i in range(len(embeddings))]
#           # embeddings = np.array(embeddings, dtype=np.float32)
#           # #print('NEW NEW embeddings:', embeddings)
          
#           return embeddings
    
#         def __len__(self):
#             return self.data.shape[0]

In [ ]:
# #vibo: два и более пропущенных слова
# #vibo: если ничего не менять получается как бы задвоение контекста, т.е.
# #vibo: когда первое слово уже заменено, при расчете контекста для второго
# #vibo: учитыается обновленная строка

# #vibo: много вариантов ошибок нужно отлавливать, просто не получается
# a = np.array(['0'], dtype='<U13')
# print(type(a))
# simple_list = [a]
# print('START simple_list', simple_list)
# '''
# #vibo:
# Исхожу из того, что нужно смотреть 2-слова-контекста справа и слева от
# слова, которого нет в словаре

# '''
# #vibo: к этому моменту имеем списки вида [..., ..., 'no_embeddings', ..., ...]

# for n in range(len(simple_list)):
#   context = []
#   #vibo: итерируемся по предложению
#   print('n:', n)
#   print('simple_list[n]:', simple_list[n])
#   #vibo: находим слово, которого нет в словаре (ранее его менял на 'no_embeddings')
#   if simple_list[n] == 'no_embeddings':
    
#     #vibo: проверяем есть ли второе слева контекствное слово
#     try:
#       if (n-2) >= 0:
#         context_word_n2_left =  simple_list[n-2]
#         print('context_word_n2_left:', context_word_n2_left)
#         if context_word_n2_left != 'no_embeddings':
#           context.append(context_word_n2_left)
#       else:
#         pass
#     except:
#       pass
   
#     #vibo: проверяем есть ли первое слева контекствное слово
#     try:
#       if (n-1) >=0:
#         context_word_n1_left = simple_list[n-1]
#         print('context_word_n1_left:', context_word_n1_left)
#         if context_word_n1_left != 'no_embeddings':
#           context.append(context_word_n1_left)
#       else:
#         pass
#     except:
#       pass
    
#     #vibo: проверяем есть ли первое справа контекствное слово
#     try:
#       context_word_n1_right = simple_list[n+1]
#       print('context_word_n1_right:', context_word_n1_right)
#       if context_word_n1_right != 'no_embeddings':
#         context.append(context_word_n1_right)
#     except:
#       pass
    
#     #vibo: проверяем есть ли второе справа контекствное слово
#     try:
#       context_word_n2_right = simple_list[n+2]
#       print('context_word_n2_right:', context_word_n2_right)
#       if context_word_n2_right != 'no_embeddings':
#         context.append(context_word_n2_right)
#     except:
#       pass

#     simple_list[n] = sum(context)
#     print('context:', context)
#     print('new simple list:', simple_list)
    
# print('FINISH simpe list:', simple_list)

<class 'numpy.ndarray'>
START simple_list [array(['0'], dtype='<U13')]
n: 0
simple_list[n]: ['0']
FINISH simpe list: [array(['0'], dtype='<U13')]


In [ ]:
#vibo: делаем новый класс от TwitterDataset, меняю get_embeddings_
class NewTwitterDataset(TwitterDataset):
        #vibo: контсруктор класса как у родителя
        def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
          #vibo: унаследуем все параметры от родителя
          super().__init__(data, feature_column, target_column, word2vec)
      
        #vibo: уточним get_tokens_
        def get_tokens_(self, text):
          # Получи все токены из текста и профильтруй их       

          #vibo: убираем имена пользователей и ссылки на сайты
          text = text.split(' ')
          text = (' '.join(filter(lambda word: not (word.lower().startswith('@') or word.lower().startswith('http')), text)))

          #vibo: полученную строку пропускаем через токенайзер, предварительно переведя в нижний регистр
          line = self.tokenizer.tokenize(text.lower())
          #vibo: удаляем знаки пунктуации и слова короче 4 букв
          tokens = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
          #vibo: возвращаем токены
          return tokens

        #vibo: уточним get_embeddings_
        def get_embeddings_(self, tokens):
        
          embeddings = []
          for w in tokens:

            try:
              word_embeddings = (self.word2vec.get_vector(w) - self.mean) / self.std
              embeddings.append(word_embeddings) 

            except:
              embeddings.append('no_embeddings')
          #print('embeddings:', embeddings)

          #vibo: надо проверить если все слова твита не в словаре
          for i in embeddings:
            if i != 'no_embeddings':
              #vibo: вызываем новую функцию
              self.new_get_embeddings_(embeddings)
              #print('embeddings:', embeddings)
            else:
              embeddings = np.zeros((1, self.word2vec.vector_size))


          if len(embeddings) == 0:
              embeddings = np.zeros((1, self.word2vec.vector_size))
          else:
              embeddings = np.array(embeddings)
              if len(embeddings.shape) == 1:
                  embeddings = embeddings.reshape(-1, 1)

          return embeddings
        
        #vibo: новая функция для твитов со словами не из словаря (суммируем контекст)
        def new_get_embeddings_(self, embeddings):
          for n in range(len(embeddings)):
            context = []
            #print('n:', n)
            #vibo: итерируемся по предложению
            #vibo: находим слово, которого нет в словаре (ранее его менял на 'no_embeddings')
            if embeddings[n] == 'no_embeddings':
              
              #vibo: проверяем есть ли второе слева контекствное слово
              try:
                if (n-2) >= 0:
                  context_word_n2_left =  embeddings[n-2]
                  if context_word_n2_left != 'no_embeddings':
                    context.append(context_word_n2_left)                  
                else:
                  pass
              except:
                pass
            
              #vibo: проверяем есть ли первое слева контекствное слово
              try:
                if (n-1) >=0:
                  context_word_n1_left = embeddings[n-1]
                  if context_word_n1_left != 'no_embeddings':
                    context.append(context_word_n1_left)
                else:
                  pass
              except:
                pass
              
              #vibo: проверяем есть ли первое справа контекствное слово
              try:
                context_word_n1_right = embeddings[n+1]
                if context_word_n1_right != 'no_embeddings':
                  context.append(context_word_n1_right)
              except:
                pass
              
              #vibo: проверяем есть ли второе справа контекствное слово
              try:
                context_word_n2_right = embeddings[n+2]
                if context_word_n2_right != 'no_embeddings':
                  context.append(context_word_n2_right)
              except:
                pass
              
              #print('context:', context)
              embeddings[n] = sum(context)
              #print('sum(context) или embeddings[n]:', embeddings[n])

          #vibo: НЕ ПРАВИЛЬНЫЙ РАЗМЕР ЭМБЕДИНГОВ без добавления
          #print('NEW embeddings:', embeddings)
          embeddings = [np.squeeze(embeddings[i][0]) for i in range(len(embeddings))]
          embeddings = np.array(embeddings, dtype=np.float32)
          #print('NEW NEW embeddings:', embeddings)

          return embeddings        

        def __len__(self):
            return self.data.shape[0]

# !!! НОВЫЙ обучающий датасет dev (dev_data)

In [ ]:
import warnings

warnings.filterwarnings('ignore')

In [ ]:
#vibo: получаем обработанный датасет
'''
dev_data - дафрейм для обучения dev_data
"text" - наименование столбца с твитами
"emotion" - наименование столбца с меткой класса
word2vec - предобученные эмбединги слов
'''

dev = NewTwitterDataset(dev_data, "text", "emotion", word2vec)

In [ ]:
for i in range(0, 5000):
  print(i)
  print((dev[i]["feature"]).shape)
  break
  #print((dev[i]["feature"]))

0
(12, 300)


In [ ]:
#vibo: проверка твитов (матрица с эмбедингами слов, входящих в твит)
(dev[4]["feature"]).shape

(6, 300)

In [ ]:
#vibo: целевая метка для соответствующего твита
dev[4]["target"]

1

# !!! Обновляем train_loader (0.8dev), valid_loader (0.2dev)

In [ ]:
from torch.utils.data import DataLoader

batch_size = 1024
#vibo: спасибо чату https://t.me/dls_spring2022_part2
#num_workers = 4

#vibo: усредняем эмбединг по строке
def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}

train_size = math.ceil(len(dev) * 0.8)
#vibo: делим обучающий датасет на train и valid 
train, valid = random_split(dev, [train_size, len(dev) - train_size])

# train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
# valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

#vibo: внимательно для train_loader shuffle=True
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, drop_last=True, collate_fn=average_emb)
#vibo: внимательно для valid_loader shuffle=False
valid_loader = DataLoader(valid, batch_size=batch_size, shuffle=False, drop_last=False, collate_fn=average_emb)

# !!! ОБУЧЕНИЕ МОДЕЛИ с обновленными данными

In [ ]:
#vibo: здесь обучаем и валидируем модель на dev(dev_data): train_loader (0.8dev) и valid_loader (0.2dev)
#vibo: ПОДБИРАЕМ ПАРАМЕТРЫ МОДЕЛИ

#vibo: бесконечность...
best_metric = np.inf

#vibo: проходим по эпохам
for e in range(num_epochs):
    #vibo: обучение, функция training на train_loader -> 0.8dev(dev_data) 
    training(model, optimizer, criterion, train_loader, e, device)
    #vibo: оцениваем модель, функция testing на valid_loader -> 0.2dev(dev_data)
    log = testing(model, criterion, valid_loader, device)
    print('Полный log (на валидации):', log)
    #vibo: сравниваем результат по loss, сохраняем лучшую модель
    if log["Test Loss"] < best_metric:
        
        
        print('Текущий Test Loss:', log["Test Loss"])
        print('Предыдущий Test Loss:', best_metric)
        
        #vibo: сохраняем модель
        torch.save(model.state_dict(), "model_context.pt")
        best_metric = log["Test Loss"]
        print('Новый Test Loss:', best_metric)
        print('')

Epoch 1. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

Полный log (на валидации): {'Test Loss': 0.5010401865243912, 'Test Acc': 0.74812890625}
Текущий Test Loss: 0.5010401865243912
Предыдущий Test Loss: inf
Новый Test Loss: 0.5010401865243912



Epoch 2. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

...

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

Полный log (на валидации): {'Test Loss': 0.5009940869808197, 'Test Acc': 0.74854296875}


Epoch 5. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

Полный log (на валидации): {'Test Loss': 0.5019097927808761, 'Test Acc': 0.74665234375}


# !!! Обновляем тестовый датасет, test_loader (test_data)

In [ ]:
#vibo: здесь оцениваем результат лучшей модели  на отложенных данных test_data(0.2data)
#vibo: ОЦЕНИВАЕМ ИТОГОВЫЙ РЕЗУЛЬТАТ

#vibo: получаем тестовый датасет (test_loader)
#vibo: внимательно для test_loader shuffle=False
#vibo: collate_fn=average_emb, average_emb - функция усреднения эмбедингов, base-line выше
test_loader = DataLoader(
    NewTwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    #num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

#vibo: загружаем лучшую сохраненную модель
model.load_state_dict(torch.load("model_context.pt", map_location=device))

#vibo: оцениваем модель, функция testing на test_loader -> test_data
print('Полный log (на тесте):', testing(model, criterion, test_loader, device=device))

Test Loss: 0, Test Acc: 0:   0%|          | 0/313 [00:00<?, ?it/s]

Полный log (на тесте): {'Test Loss': 0.5076717542002376, 'Test Acc': 0.7431047823482428}


# !!! ЗАДАНИЕ-3 TFIDF

In [19]:
#vibo: данные из для получения tfidf
dev_data

,emotion,id,date,flag,user,text
0,0,1564500154,Mon Apr 20 03:47:08 PDT 2009,NO_QUERY,zourzouvillys,@Claire_Nelson i'm on the north devon coast th...
1,4,1957039896,Thu May 28 23:21:01 PDT 2009,NO_QUERY,twinkleval,@jhicks i will think of you on Sunday! Who ...
2,4,1557601862,Sun Apr 19 05:03:53 PDT 2009,NO_QUERY,andrewbulloch,Out in the garden with the kids debating wheth...
3,4,1823599026,Sat May 16 22:25:01 PDT 2009,NO_QUERY,vikchopra,@FrVerona thank u my love...u've shown me the ...
4,4,2186814798,Mon Jun 15 19:22:35 PDT 2009,NO_QUERY,jobrofan16,is with @jonasbrosfan1 going to buy LVATT tog...
...,...,...,...,...,...,...
1279995,4,1977221349,Sat May 30 20:25:09 PDT 2009,NO_QUERY,omgxregina,@yajtyler thats so sweet =] they like you and...
1279996,0,2178617823,Mon Jun 15 07:57:56 PDT 2009,NO_QUERY,ilykthx,Really sad the NBA playoffs are over for good
1279997,4,2058796673,Sat Jun 06 15:38:43 PDT 2009,NO_QUERY,Socallmelovely,@JuJuBeanz15 LOL Im glad that u have that pers...
1279998,0,1835648576,Mon May 18 06:27:33 PDT 2009,NO_QUERY,tercowas,I have to go to the GYM it's toooooo hot.


In [20]:
dev_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1280000 entries, 0 to 1279999
Data columns (total 6 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   emotion  1280000 non-null  int64 
 1   id       1280000 non-null  int64 
 2   date     1280000 non-null  object
 3   flag     1280000 non-null  object
 4   user     1280000 non-null  object
 5   text     1280000 non-null  object
dtypes: int64(2), object(4)
memory usage: 58.6+ MB


In [ ]:
# #vibo: удаляем html элементы
# dev_data['clean_text'] = dev_data['text'].str.replace(r"<[a-z/]+>", " ") 
# #vibo: удаляем специальные символы и цифры
# dev_data['clean_text'] = dev_data['clean_text'].str.replace(r"[^A-Za-z]+", " ") 

In [21]:
dev_data['text'][0]

"@Claire_Nelson i'm on the north devon coast the next few weeks  will be down in Devon again in may sometime i hope though!"

In [ ]:
# dev_data['clean_text'][0]

In [22]:
#vibo: импортируем векторайзер
from sklearn.feature_extraction.text import TfidfVectorizer

In [23]:
#vibo: создаем объект
vectorizer = TfidfVectorizer(analyzer='word', stop_words='english')

In [24]:
corpus_tfidf = vectorizer.fit_transform(dev_data['text'])

In [25]:
#vibo: размер (c исключением stop-words)
print(corpus_tfidf.shape)

(1280000, 589294)


In [26]:
#vibo: '589294' слов из словаря
print(vectorizer.get_feature_names_out()[100000:100100])

['bridget_the1' 'bridgetamelia' 'bridgetayers' 'bridgetbehm25'
 'bridgetboston88' 'bridgetcarey' 'bridgetchumbley' 'bridgetd'
 'bridgetdginley' 'bridgetdill' 'bridgetdooley' 'bridgetem'
 'bridgetforney' 'bridgethaddix' 'bridgetjayne' 'bridgetkulak'
 'bridgetlaffy' 'bridgetlee_ob' 'bridgetlocke' 'bridgetmckenzie'
 'bridgetmcmanus' 'bridgetnegron' 'bridgetneval' 'bridgeton'
 'bridgetparsons' 'bridgetphoto' 'bridgets' 'bridgetsbeaches'
 'bridgetschuberg' 'bridgetslater' 'bridgetsupple' 'bridgett' 'bridgette'
 'bridgetteb' 'bridgettebanks' 'bridgettebanxxx' 'bridgettegreen'
 'bridgetteo' 'bridgettes' 'bridgettesharp' 'bridgetwinder' 'bridgetx'
 'bridgetzinn' 'bridgeview' 'bridgewater' 'bridgeway' 'bridgeykate'
 'bridgeymah' 'bridgford' 'bridght' 'bridgie23' 'bridging' 'bridgit'
 'bridgnorth' 'bridgwater' 'bridie' 'bridie_slade93' 'bridiemacdonald'
 'bridiemay' 'bridiestearn' 'bridkeogh' 'bridle' 'bridles' 'bridlington'
 'bridofprey' 'bridog0' 'bridold' 'bridport' 'bridsmaid' 'bridugan'
 'b

In [27]:
print(corpus_tfidf[0])

  (0, 240698)	0.16644664633562847
  (0, 560858)	0.22358481141812767
  (0, 130265)	0.2909832262667077
  (0, 157956)	0.716740637259165
  (0, 388985)	0.28780226471949644
  (0, 127496)	0.49100593008823756


In [28]:
print(corpus_tfidf[0, 240698])

0.16644664633562847


In [29]:
from sklearn.decomposition import IncrementalPCA
from scipy import sparse

In [30]:
transformer = IncrementalPCA(n_components=300, batch_size=1000)

In [31]:
X_sparse = sparse.csr_matrix(corpus_tfidf)

In [ ]:
X_transformed = transformer.fit_transform(X_sparse)

# !!! TfidfTwitterDataset

In [ ]:
#vibo: делаем новый класс от TwitterDataset, меняю get_embeddings_
class TfidfTwitterDataset(TwitterDataset):
        #vibo: конструктор класса как у родителя
        def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
          #vibo: унаследуем все параметры от родителя
          super().__init__(data, feature_column, target_column, word2vec)
           
        #vibo: уточняем get_tokens_
        def get_tokens_(self, text):
            # Получи все токены из текста и профильтруй их
          
            #vibo: убираем имена пользователей и ссылки на сайты
            text = text.split(' ')
            text = (' '.join(filter(lambda word: not (word.lower().startswith('@') or word.lower().startswith('http')), text)))

            #vibo: полученную строку пропускаем через токенайзер, предварительно переведя в нижний регистр
            line = self.tokenizer.tokenize(text.lower())
            #print('line:', line)
            #vibo: удаляем знаки пунктуации и слова короче 4 букв
            tokens = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
            #vibo: возвращаем токены
            return  tokens



        
        #vibo: получаем tfidf слов
        def get_tfidf_(self, text):

            tfidf = 1_000_000

            return  tfidf   
        


        #vibo: уточняем get_embeddings_
        def get_embeddings_(self, tokens):
          
            embeddings = []
            for w in tokens:

              try:
                word_embeddings = (self.word2vec.get_vector(w) - self.mean) / self.std
                embeddings.append(word_embeddings) 

              except:
                embeddings.append('no_embeddings')
            #print('embeddings:', embeddings)

            #vibo: надо проверить если все слова твита не в словаре
            for i in embeddings:
              if i != 'no_embeddings':
                #vibo: вызываем новую функцию
                self.new_get_embeddings_(embeddings)
                #print('embeddings:', embeddings)
              else:
                embeddings = np.zeros((1, self.word2vec.vector_size))

            if len(embeddings) == 0:
                embeddings = np.zeros((1, self.word2vec.vector_size))
            else:
                embeddings = np.array(embeddings)
                if len(embeddings.shape) == 1:
                    embeddings = embeddings.reshape(-1, 1)

            return embeddings

          
        def __len__(self):
            return self.data.shape[0]

# !!! НОВЫЙ обучающий датасет c Tfidf dev (dev_data)

In [ ]:
#vibo: получаем обработанный датасет
'''
dev_data - дафрейм для обучения dev_data
"text" - наименование столбца с твитами
"emotion" - наименование столбца с меткой класса
word2vec - предобученные эмбединги слов
'''

dev = TfidfTwitterDataset(dev_data, "text", "emotion", word2vec)

In [ ]:
for i in range(0, 5000):
  print(i)
  print((dev[i]["feature"]).shape)
  break
  #print((dev[i]["feature"]))

In [ ]:
#vibo: проверка твитов (матрица с эмбедингами слов, входящих в твит)
(dev[4]["feature"]).shape

In [ ]:
#vibo: целевая метка для соответствующего твита
dev[4]["target"]

# !!! Обновляем train_loader (0.8dev), valid_loader (0.2dev)

In [ ]:
from torch.utils.data import DataLoader

batch_size = 1024
#vibo: спасибо чату https://t.me/dls_spring2022_part2
#num_workers = 4

#vibo: усредняем эмбединг по строке
def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}

train_size = math.ceil(len(dev) * 0.8)
#vibo: делим обучающий датасет на train и valid 
train, valid = random_split(dev, [train_size, len(dev) - train_size])

# train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
# valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

#vibo: внимательно для train_loader shuffle=True
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, drop_last=True, collate_fn=average_emb)
#vibo: внимательно для valid_loader shuffle=False
valid_loader = DataLoader(valid, batch_size=batch_size, shuffle=False, drop_last=False, collate_fn=average_emb)

# !!! ОБУЧЕНИЕ МОДЕЛИ с обновленными данными

In [ ]:
#vibo: здесь обучаем и валидируем модель на dev(dev_data): train_loader (0.8dev) и valid_loader (0.2dev)
#vibo: ПОДБИРАЕМ ПАРАМЕТРЫ МОДЕЛИ

#vibo: бесконечность...
best_metric = np.inf

#vibo: проходим по эпохам
for e in range(num_epochs):
    #vibo: обучение, функция training на train_loader -> 0.8dev(dev_data) 
    training(model, optimizer, criterion, train_loader, e, device)
    #vibo: оцениваем модель, функция testing на valid_loader -> 0.2dev(dev_data)
    log = testing(model, criterion, valid_loader, device)
    print('Полный log (на валидации):', log)
    #vibo: сравниваем результат по loss, сохраняем лучшую модель
    if log["Test Loss"] < best_metric:
        
        
        print('Текущий Test Loss:', log["Test Loss"])
        print('Предыдущий Test Loss:', best_metric)
        
        #vibo: сохраняем модель
        torch.save(model.state_dict(), "model_tfidf.pt")
        best_metric = log["Test Loss"]
        print('Новый Test Loss:', best_metric)
        print('')

# !!! Обновляем тестовый датасет, test_loader (test_data)

In [ ]:
#vibo: здесь оцениваем результат лучшей модели  на отложенных данных test_data(0.2data)
#vibo: ОЦЕНИВАЕМ ИТОГОВЫЙ РЕЗУЛЬТАТ

#vibo: получаем тестовый датасет (test_loader)
#vibo: внимательно для test_loader shuffle=False
#vibo: collate_fn=average_emb, average_emb - функция усреднения эмбедингов, base-line выше
test_loader = DataLoader(
    NewTwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    #num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

#vibo: загружаем лучшую сохраненную модель
model.load_state_dict(torch.load("model_tfidf.pt", map_location=device))

#vibo: оцениваем модель, функция testing на test_loader -> test_data
print('Полный log (на тесте):', testing(model, criterion, test_loader, device=device))

In [ ]:
#vibo: Виталий_Бочаров_stepik_id_56089805, telegram(@VitalyBo), 27.04.2022